# BMI565: Bioinformatics Programming & Scripting

#### (C) Michael Mooney (mooneymi@ohsu.edu)

## Week 8: BioPython - Alignments and BLAST

1. Sequence Alignment
2. MUSCLE
    - Alignment I/O
    - `AlignInfo` module
3. BLAST
    - BLAST Queries Using BioPython

#### Requirements

- Python 2.7 or 3.x
- `Bio` (BioPython) module (`conda install biopython`)
- MUSCLE command-line program 
    - [https://github.com/rcedgar/muscle](https://github.com/rcedgar/muscle)
- Data Files
    - `./data/egfr.fasta`

In [1]:
from __future__ import print_function, division

## Sequence Alignment

By aligning biological sequences we can identify functional, structural, or evolutionary relationships between sequences. 

- Pairwise alignment algorithms compare two sequences
    - Smith-Waterman
    - Needleman-Wunsch
- Multiple sequence alignment (MSA) algorithms compare two or more sequences
    - clustalw
    - muscle
    - tcoffee
    
BioPython supports both types of alignment through its `Align` module.

## MUSCLE

#### Installing MUSCLE: 

For those using a Mac, run the following: `conda install -c bioconda muscle`

Windows users can download a pre-compiled executable here: [https://github.com/rcedgar/muscle/releases/tag/v5.1](https://github.com/rcedgar/muscle/releases/tag/v5.1)

MUSCLE Documentation: [https://drive5.com/muscle5/manual/index.html](https://drive5.com/muscle5/manual/index.html)

The MUSCLE MSA program uses progressive alignment construction by taking the following steps:

1. Perform pair-wise alignment of all sequences
2. Construct a tree such that branches represent the most similar sequences
3. Collapse branches into groups in order of similarity 

The algorithm implements three stages (draft progressive, improved progressive, and refinement) to arrive at the final alignment. Details of the algorithm can be found in the following paper:

[https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-5-113](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-5-113)

In [2]:
## Wrappers for command-line programs, like MUSCLE, are being deprecated
## https://github.com/biopython/biopython/issues/2877
#from Bio.Align.Applications import MuscleCommandline
import subprocess

In [3]:
## Create and run MUSCLE command
muscle_result = subprocess.run(["muscle", "-align", "./data/egfr.fasta", "-output", "./data/egfr.afa"], capture_output=True)

In [4]:
muscle_result.returncode

0

In [5]:
print(muscle_result.stderr.decode())


muscle 5.1.osx64 []  17.2Gb RAM, 8 cores
Built Feb 22 2022 02:38:35
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 4 seqs, avg length 737, max 1210

00:00 4.2Mb  CPU has 8 cores, running 8 threads
00:00 4.7Mb   100.0% Calc posteriors
00:00 134Mb   100.0% Consistency (1/2)
00:00 134Mb   100.0% Consistency (2/2)
00:00 134Mb   100.0% UPGMA5
00:00 118Mb   100.0% Refining



### Alignment I/O

BioPython stores the results of a MSA in `MultipleSeqAlignment` objects. These objects can be created by reading the results of an alignment from a file using the `AlignIO` module.

Supported formats: [http://biopython.org/wiki/AlignIO](http://biopython.org/wiki/AlignIO)

In [6]:
from Bio import AlignIO

## Read the alignment results from the ClustalW output file
egfr_alignment = AlignIO.read("./data/egfr.afa", "fasta")

## egfr_alignment is now a BioPython MultipleSeqAlignment object
print(egfr_alignment)

Alignment with 4 rows and 1214 columns
MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTF...SCH sp|P00533-3|EGFR_HUMAN
MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTF...--- sp|P00533-2|EGFR_HUMAN
MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTF...--- sp|P00533-4|EGFR_HUMAN
MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTF...--- sp|P00533|EGFR_HUMAN


In [7]:
for seq in egfr_alignment:
    print(seq.seq[1209])
    

A
-
-
A


In [8]:
help(egfr_alignment)

Help on MultipleSeqAlignment in module Bio.Align object:

class MultipleSeqAlignment(builtins.object)
 |  MultipleSeqAlignment(records, alphabet=None, annotations=None, column_annotations=None)
 |  
 |  Represents a classical multiple sequence alignment (MSA).
 |  
 |  By this we mean a collection of sequences (usually shown as rows) which
 |  are all the same length (usually with gap characters for insertions or
 |  padding). The data can then be regarded as a matrix of letters, with well
 |  defined columns.
 |  
 |  You would typically create an MSA by loading an alignment file with the
 |  AlignIO module:
 |  
 |  >>> from Bio import AlignIO
 |  >>> align = AlignIO.read("Clustalw/opuntia.aln", "clustal")
 |  >>> print(align)
 |  Alignment with 7 rows and 156 columns
 |  TATACATTAAAGAAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273285|gb|AF191659.1|AF191
 |  TATACATTAAAGAAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273284|gb|AF191658.1|AF191
 |  TATACATTAAAGAAGGGGGATGCGGATAAATGGAAAGGCGAAA

In [9]:
## MultipleSeqAlignment objects are iterable objects with one element per sequence
len(egfr_alignment)

4

In [10]:
## iterate through an alignment object
for seq in egfr_alignment:
    print(seq.seq)

MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGPGNESLKA------------------------------------------------------------------ML--------------------------------------------------------------------------------------------FCLFK--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
## Use indices to retrieve sequences
egfr_alignment[0]

SeqRecord(seq=Seq('MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRM...SCH'), id='sp|P00533-3|EGFR_HUMAN', name='sp|P00533-3|EGFR_HUMAN', description='sp|P00533-3|EGFR_HUMAN Isoform 3 of Epidermal growth factor receptor OS=Homo sapiens GN=EGFR', dbxrefs=[])

In [12]:
## Use slice notation to retrieve columns of the alignment
## For instance, to see agreement at a specific location
egfr_alignment[:,750]

'---T'

In [13]:
print(egfr_alignment[:,750:775])

Alignment with 4 rows and 25 columns
------------------------- sp|P00533-3|EGFR_HUMAN
------------------------- sp|P00533-2|EGFR_HUMAN
------------------------- sp|P00533-4|EGFR_HUMAN
TSPKANKEILDEAYVMASVDNPHVC sp|P00533|EGFR_HUMAN


#### Reading and Writing Alignments

Similar to SeqIO, we can read very large alignment files using a generator returned from the `AlignIO.parse()` method. Alignments can also be written to file in a variety of formats.

In [14]:
## Use a generator to read an alignment file
alignments = AlignIO.parse("./data/egfr.afa", "fasta")

for alignment in alignments:
    print(alignment)

Alignment with 4 rows and 1214 columns
MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTF...SCH sp|P00533-3|EGFR_HUMAN
MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTF...--- sp|P00533-2|EGFR_HUMAN
MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTF...--- sp|P00533-4|EGFR_HUMAN
MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTF...--- sp|P00533|EGFR_HUMAN


In [15]:
## Write an alignment to a file
AlignIO.write(egfr_alignment, "example.stk", "stockholm")

1

### `AlignInfo` module

The `AlignInfo` module allows the generation of a consensus sequence resulting from an alignment. The consensus sequence contains the most common letter at each sequence position.

`AlignInfo` contains two methods for generating a consensus sequence:

`dumb_consensus()` counts the number of matches at a particular position. If a letter count exceeds a threshold it is included in the consensus, otherwise an ambiguous character is included.

`gap_consensus()` is the same as `dumb_consensus()` except that it allows gaps.

In [16]:
from Bio.Align import AlignInfo

In [17]:
summary = AlignInfo.SummaryInfo(egfr_alignment)

In [18]:
help(summary.dumb_consensus)

Help on method dumb_consensus in module Bio.Align.AlignInfo:

dumb_consensus(threshold=0.7, ambiguous='X', require_multiple=False) method of Bio.Align.AlignInfo.SummaryInfo instance
    Output a fast consensus sequence of the alignment.
    
    This doesn't do anything fancy at all. It will just go through the
    sequence residue by residue and count up the number of each type
    of residue (ie. A or G or T or C for DNA) in all sequences in the
    alignment. If the percentage of the most common residue type is
    greater then the passed threshold, then we will add that residue type,
    otherwise an ambiguous character will be added.
    
    This could be made a lot fancier (ie. to take a substitution matrix
    into account), but it just meant for a quick and dirty consensus.
    
    Arguments:
     - threshold - The threshold value that is required to add a particular
       atom.
     - ambiguous - The ambiguous character to be added when the threshold is
       not reached.


In [19]:
summary.dumb_consensus(threshold=0.6, ambiguous='X')

Seq('MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRM...SCH')

In [20]:
## View the dumb_consensus sequence
str(summary.dumb_consensus(threshold=0.6, ambiguous='X'))

'MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGXXXXXLXXCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQXLLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFXCLXXYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYR

In [21]:
## View the gap_consensus sequence
str(summary.gap_consensus(threshold=0.6, ambiguous='X'))

'MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGXXXXXXXX------------------------------------------------------------------XX--------------------------------------------------------------------------------------------XXXXX-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## BLAST: Basic Local Alignment Search Tool

- Developed in 1990 by researchers at NCBI, Penn State, and University of Arizona (biologists and computer scientists)
- Emphasizes speed over accuracy, which is necessary when searching databases containing hundreds of millions of sequences
- [http://blast.ncbi.nlm.nih.gov/Blast.cgi](http://blast.ncbi.nlm.nih.gov/Blast.cgi)

#### The BLAST Algorithm

1. Break query sequence into "words" (substrings)
2. Search a list of indexed words for similarity
3. Use thresholding to eliminate poor matches
4. Search database for sequences associated with high-scoring "words"
5. Extend matching alignments and score the sequence's similarity to quantify high-scoring sequence pairs (HSPs)
6. Combine HSPs if mapping to the same database sequence
7. Display scores and Smith-Waterman alignment for high-scoring matches

The results of the algorithm is an E-value for each matching sequence. The E-value represents the probability of getting the alignment by chance (a p-value corrected for multiple testing).

Details about the sequence similarity scores can be found here: [http://www.ncbi.nlm.nih.gov/BLAST/tutorial/Altschul-1.html](http://www.ncbi.nlm.nih.gov/BLAST/tutorial/Altschul-1.html)

### BLAST Queries Using BioPython

BioPython supports running BLAST queries either locally or over the internet. To run BLAST locally you will need to install the algorithm and sequence databases on your machine. The advantages of running queries locally are improved performance (faster) and customizable databases. Use the following module for local BLAST:

    from Bio.Blast.Applications import NcbiblastpCommandline
    
We will focus on running BLAST queries over the internet, which requires less initial setup and provides access to NCBI's sequence databases. The `qblast()` method in the `Bio.Blast.NCBIWWW` module can be used to submit BLAST queries. The method takes three parameters:

1. The blast program (indicates the type of query)
    - blastp: amino acid sequence queried against protein database
    - blastn: nucleotide sequence queried against nucleotide database
    - blastx: translated nucleotide sequence queried against protein database
2. The database to search
    - [https://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf](https://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf)
3. The query sequence (a string)

The `qblast()` method returns XML formatted results.

In [22]:
from Bio.Blast import NCBIWWW

In [23]:
## Get a query sequence
seq = str(summary.dumb_consensus(threshold=0.6, ambiguous='X'))
seq

'MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGXXXXXLXXCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQXLLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFXCLXXYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYR

In [24]:
## WARNING -- this may take a while to run
## Run a BLAST query
result_handle = NCBIWWW.qblast('blastp', 'nr', seq)

## Write XML results to file
fh = open("blast_results.xml", 'w')
fh.write(result_handle.read())
fh.close()

#### Reading BLAST XML Results

In [25]:
## Import module for parsing BLAST XML results
from Bio.Blast import NCBIXML

## Read BLAST results
fh = open('blast_results.xml')
blast_record = NCBIXML.read(fh)
fh.close()

In [26]:
## Get descriptions of matches
len(blast_record.descriptions)

50

In [27]:
for description in blast_record.descriptions[0:3]:
    print(description.title)
    print(description.score)
    print(description.e, '\n')

ref|NP_005219.2| epidermal growth factor receptor isoform a precursor [Homo sapiens] >sp|P00533.2| RecName: Full=Epidermal growth factor receptor; AltName: Full=Proto-oncogene c-ErbB-1; AltName: Full=Receptor tyrosine-protein kinase erbB-1; Flags: Precursor [Homo sapiens] >gb|AAX41052.1| epidermal growth factor receptor [synthetic construct] >gb|AAG35789.1| p170 epidermal growth factor receptor [Homo sapiens] >gb|AAS83109.1| epidermal growth factor receptor (erythroblastic leukemia viral (v-erb-b) oncogene homolog, avian) [Homo sapiens] >gb|AAX41053.1| epidermal growth factor receptor [synthetic construct] >gb|EAW50962.1| epidermal growth factor receptor (erythroblastic leukemia viral (v-erb-b) oncogene homolog, avian), isoform CRA_a [Homo sapiens]
5989.0
0.0 

dbj|BAI46646.1| epidermal growth factor receptor, partial [synthetic construct]
5987.0
0.0 

pdb|7SYD|A Chain A, Epidermal growth factor receptor [Homo sapiens] >pdb|7SYD|B Chain B, Epidermal growth factor receptor [Homo sapiens

In [28]:
## Get alignment info about matches
len(blast_record.alignments)

50

In [29]:
for alignment in blast_record.alignments[0:2]:
    print(alignment.title)
    for hsp in alignment.hsps:
        print(hsp.score)
        print("Query: ", hsp.query, '\n')
        print("Match: ", hsp.match, '\n')
        print("Subject: ", hsp.sbjct, '\n')

ref|NP_005219.2| epidermal growth factor receptor isoform a precursor [Homo sapiens] >sp|P00533.2| RecName: Full=Epidermal growth factor receptor; AltName: Full=Proto-oncogene c-ErbB-1; AltName: Full=Receptor tyrosine-protein kinase erbB-1; Flags: Precursor [Homo sapiens] >gb|AAX41052.1| epidermal growth factor receptor [synthetic construct] >gb|AAG35789.1| p170 epidermal growth factor receptor [Homo sapiens] >gb|AAS83109.1| epidermal growth factor receptor (erythroblastic leukemia viral (v-erb-b) oncogene homolog, avian) [Homo sapiens] >gb|AAX41053.1| epidermal growth factor receptor [synthetic construct] >gb|EAW50962.1| epidermal growth factor receptor (erythroblastic leukemia viral (v-erb-b) oncogene homolog, avian), isoform CRA_a [Homo sapiens]
5989.0
Query:  MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGK

## References

- Python for Bioinformatics, Sebastian Bassi, CRC Press (2010)
- [http://biopython.org/DIST/docs/tutorial/Tutorial.html](http://biopython.org/DIST/docs/tutorial/Tutorial.html)
- [http://biopython.org/DIST/docs/api/](http://biopython.org/DIST/docs/api/)
- Peter Cock et al. Biopython: freely available Python tools for computational molecular biology and bioinformatics, <i>Bioinformatics</i> (2009)

#### Last Updated: 19-Sep-2022